In [ ]:
import pandas as pd
import pycountry

# IDI Scores

# Load CSV into a Pandas DataFrame
idi_file_path = "IDI_Scores.csv" 
df_tech = pd.read_csv(idi_file_path)

# Rename the first column to "Country"
df_tech.rename(columns={df_tech.columns[0]: "Country"}, inplace=True)
df_tech.rename(columns={df_tech.columns[1]: "Tech"}, inplace=True)

df_tech.reset_index(drop=True, inplace=True)


# Health Scores

# Load CSV into a Pandas DataFrame
ghsi_file_path = "2021-GHS-Index-April-2022.csv"  
df_health = pd.read_csv(ghsi_file_path)

df_health = df_health[["Country", "Year", "OVERALL SCORE"]]
df_health.rename(columns={df_health.columns[2]: "Health"}, inplace=True)

df_health = df_health[df_health["Year"] == 2021]

df_health = df_health[["Country", "Health"]]

df_health.reset_index(drop=True, inplace=True)


# Education Scores

# Load CSV into a Pandas DataFrame
lit_file_path = "school.csv" 
df_edu = pd.read_csv(lit_file_path)

df_edu.rename(columns={df_edu.columns[0]: "Country"}, inplace=True)
df_edu.rename(columns={df_edu.columns[3]: "Education"}, inplace=True)

# Keep only the latest year per geoUnit
df_edu = df_edu.sort_values(by=["Code", "Year"], ascending=[True, False]) 
df_edu = df_edu.drop_duplicates(subset=["Code"], keep="first")  

# Rename columns for clarity
df_edu.drop(columns=["Year"], inplace=True)
df_edu.reset_index(drop=True, inplace=True)


# Violence Scores

# Load the crime data CSV file
crime_file_path = "data_cts_violent_and_sexual_crime.csv" 
df_crime = pd.read_csv(crime_file_path)

# Filter the DataFrame to keep only the relevant categories
df_crime_filtered = df_crime[df_crime["Category"].isin(["Serious assault", "Kidnapping", "Sexual violence"])]

# Sort by Country and Year in descending order (most recent year first)
df_crime_filtered = df_crime_filtered.sort_values(by=["Country", "Year"], ascending=[True, False])

# Keep only the most recent year entry per Country and Category
df_crime_filtered = df_crime_filtered.drop_duplicates(subset=["Country", "Category"], keep="first")

# Group by Country and sum the values for selected categories
df_crime_summed = df_crime_filtered.groupby("Country", as_index=False)["VALUE"].sum()

# Rename the column to reflect the combined crime metric
df_crime_summed.rename(columns={"VALUE": "Violence"}, inplace=True)

df_crime = df_crime_summed.copy()
df_crime.reset_index(drop=True, inplace=True)


# Income Inequality

# Load CSV into a Pandas DataFrame
income_file_path = "gini-coefficient.csv"  
df_income = pd.read_csv(income_file_path)

df_income.rename(columns={"Gini coefficient (before tax) (World Inequality Database)": "Inequality"}, inplace=True)

# Sort by Country and Year in descending order (most recent year first)
df_income = df_income.sort_values(by=["Country", "Year"], ascending=[True, False])

# Keep only the most recent year entry per Country and Category
df_income = df_income.drop_duplicates(subset=["Country"], keep="first")
df_income = df_income[["Country", "Inequality"]]

# Drop rows where the "Country" column contains "(WID)"
df_income = df_income[~df_income["Country"].str.contains(r"World", na=False)]
df_income = df_income[~df_income["Country"].str.contains(r"\(WID\)", na=False)]

df_income.reset_index(drop=True, inplace=True)

# Function to convert country name to ISO3 code
def get_iso3(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return None  # Return None if no match is found

# Function to standardize location field across different dataframes
def standardize_countries(df, country_col="Country", iso3_col="Iso3_code"):
    df["ISO3"] = df[country_col].apply(get_iso3)

    return df

# Apply the function to all five datasets
df_tech = standardize_countries(df_tech, country_col="Country", iso3_col=None)  # IDI scores
df_health = standardize_countries(df_health, country_col="Country", iso3_col=None)  # Healthcare
df_edu = standardize_countries(df_edu, country_col="Country", iso3_col="Code")  # Literacy
df_crime = standardize_countries(df_crime, country_col="Country", iso3_col=None)  # Violent Crime
df_income = standardize_countries(df_income, country_col="Country", iso3_col=None)  # Poverty

# Fix Missing ISOs
manual_fixes = {
    "Brunei": "BRN",
    "Cape Verde": "CPV",
    "China (rural)": "CHN",
    "China (urban)": "CHN",
    "Cote d'Ivoire": "CIV",
    "Democratic Republic of Congo": "COD",
    "Palestine": "PSE",
    "Russia": "RUS",
    "Turkey": "TUR",
    "Bolivia (Plurinational State of)": "BOL",
    "Congo (Rep. of the)": "COG",
    "Dem. Rep. of the Congo": "COD",
    "Dominican Rep.": "",
    "Hong Kong, China": "HKG",
    "Iran (Islamic Republic of)": "IRN",
    "Korea (Rep. of)": "KOR",
    "Lao P.D.R.": "LAO",
    "Macao, China": "MAC",
    "Netherlands (Kingdom of the)": "NLD",
    "Bosnia and Hercegovina": "BIH",
    "Congo (Brazzaville)": "COG",
    "Congo (Democratic Republic)": "COD",
    "Micronesia (country)": "FSM",
    "East Timor": "TLS",
    "China, Hong Kong Special Administrative Region": "HKG",
    "China, Macao Special Administrative Region": "MAC",
    "Côte dIvoire": "CIV",
    "Holy See": "VAT",
    "Iraq (Central Iraq)": "IRQ",
    "Republic of Korea": "KOR",
    "State of Palestine": "PSE",
    "United Kingdom (England and Wales)": "GBR",
    "United Kingdom (Northern Ireland)": "GBR",
    "United Kingdom (Scotland)": "GBR",
    "Venezuela (Bolivarian Republic of)": "VEN"
}

# Apply manual fixes **only where ISO3 is still missing**
df_tech.loc[df_tech["ISO3"].isna(), "ISO3"] = df_tech["Country"].map(manual_fixes)
df_health.loc[df_health["ISO3"].isna(), "ISO3"] = df_health["Country"].map(manual_fixes)
df_edu.loc[df_edu["ISO3"].isna(), "ISO3"] = df_edu["Country"].map(manual_fixes)
df_crime.loc[df_crime["ISO3"].isna(), "ISO3"] = df_crime["Country"].map(manual_fixes)
df_income.loc[df_income["ISO3"].isna(), "ISO3"] = df_income["Country"].map(manual_fixes)

# Drop rows where ISO3 is NaN for each dataframe
df_tech = df_tech.dropna(subset=["ISO3"])
df_health = df_health.dropna(subset=["ISO3"])
df_edu = df_edu.dropna(subset=["ISO3"])
df_crime = df_crime.dropna(subset=["ISO3"])
df_income = df_income.dropna(subset=["ISO3"])

# Reset index after dropping rows 
df_tech.reset_index(drop=True, inplace=True)
df_health.reset_index(drop=True, inplace=True)
df_edu.reset_index(drop=True, inplace=True)
df_crime.reset_index(drop=True, inplace=True)
df_income.reset_index(drop=True, inplace=True)

# Function to check for duplicate ISO3 values in a dataframe
def check_duplicates(df, name):
    duplicates = df[df.duplicated(subset=["ISO3"], keep=False)]  
    if not duplicates.empty:
        print(f" Duplicate ISO3 codes found in {name}:")
        print(duplicates[["ISO3", "Country"]].sort_values(by="ISO3")) 
        print("\n" + "="*50 + "\n")
    else:
        print(f" No duplicates found in {name}.\n")

# Run duplicate checks on each dataframe
check_duplicates(df_tech, "Tech Data")
check_duplicates(df_health, "Health Data")
check_duplicates(df_edu, "Education Data")
check_duplicates(df_crime, "Crime Data")
check_duplicates(df_income, "Income Data")

# Identify and sum the Violence values for duplicate ISO3 = "GBR"
gbr_sum = df_crime[df_crime["ISO3"] == "GBR"]["Violence"].sum()

# Create a new row for the United Kingdom with the summed value
new_row = pd.DataFrame({"ISO3": ["GBR"], "Country": ["United Kingdom"], "Violence": [gbr_sum]})

# Remove the individual rows for England, Scotland, and Northern Ireland
df_crime = df_crime[df_crime["ISO3"] != "GBR"]

# Append the new aggregated row
df_crime = pd.concat([df_crime, new_row], ignore_index=True)

# Remove the rows where ISO3 is "CHN" and the Country is "China (rural)" or "China (urban)"
df_income = df_income[~((df_income["ISO3"] == "CHN") & (df_income["Country"].isin(["China (rural)", "China (urban)"])))]

# Drop the "Country" column from each dataframe before merging
df_tech = df_tech.drop(columns=["Country"], errors="ignore")
df_health = df_health.drop(columns=["Country"], errors="ignore")
df_edu = df_edu.drop(columns=["Country","Code"], errors="ignore")
df_crime = df_crime.drop(columns=["Country"], errors="ignore")
df_income = df_income.drop(columns=["Country"], errors="ignore")

# Merge all datasets on ISO3, ensuring only complete data remains (inner join)
df_final = df_tech.merge(df_health, on="ISO3", how="inner") \
                  .merge(df_edu, on="ISO3", how="inner") \
                  .merge(df_crime, on="ISO3", how="inner") \
                  .merge(df_income, on="ISO3", how="inner")

# Reset index after merging
df_final.reset_index(drop=True, inplace=True)

# Load the scatterplot dataset
scatterplot_file_path = "scatterplot.csv" 
df_scatterplot = pd.read_csv(scatterplot_file_path)

# Function to convert country names to ISO3 codes
def get_iso3(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return None  

# Add a new column "ISO3" and apply the mapping function
df_scatterplot["ISO3"] = df_scatterplot["Country"].apply(get_iso3)

# Identify rows where ISO3 is still missing (for manual resolution)
missing_iso3 = df_scatterplot[df_scatterplot["ISO3"].isna()]

# Display missing mappings for manual correction
print("Missing ISO3 mappings:")
print(missing_iso3[["Country"]])

manual_fixes = {
    "Taiwan Province of China": "TWN",
    "Russia": "RUS",
    "Hong Kong S.A.R. of China": "HKG",
    "Congo (Brazzaville)": "COG",
    "Ivory Coast": "CIV",
    "Turkiye": "TUR",
    "Congo (Kinshasa)": "COD"
}

# Apply manual fixes where ISO3 is still missing
df_scatterplot.loc[df_scatterplot["ISO3"].isna(), "ISO3"] = df_scatterplot["Country"].map(manual_fixes)

# Identify rows where ISO3 is still missing (for manual resolution)
missing_iso3 = df_scatterplot[df_scatterplot["ISO3"].isna()]

df_scatterplot = df_scatterplot.dropna(subset=["ISO3"])
df_scatterplot= df_scatterplot[["Ladder Score","ISO3"]]

# Merge df_final with df_scatterplot on ISO3
df_final = df_final.merge(df_scatterplot, on="ISO3", how="inner")

# Reset index for cleanliness
df_final.reset_index(drop=True, inplace=True)

# Function to convert ISO3 code to full country name
def get_country_name(iso3_code):
    try:
        return pycountry.countries.get(alpha_3=iso3_code).name
    except AttributeError:
        return None 

# Create a new column "Country_Name" in df_final
df_final["Country_Name"] = df_final["ISO3"].apply(get_country_name)

# Define the desired column order
column_order = ["Country_Name", "ISO3", "Ladder Score", "Tech", "Health", "Education", "Violence", "Inequality"]

# Reorder the dataframe columns
df_final = df_final[column_order]

# Define the columns to classify
columns_to_classify = ["Tech", "Health", "Education", "Violence", "Inequality"]

# Function to assign levels (1-5) based on quantiles
def assign_levels(series):
    return pd.qcut(series, q=5, labels=[1, 2, 3, 4, 5])

# Apply classification to each column
for col in columns_to_classify:
    df_final[col + "_Level"] = assign_levels(df_final[col])

# Load a mapping of ISO3 codes to regions
region_df = pd.read_csv('https://raw.githubusercontent.com/datasets/country-codes/master/data/country-codes.csv')

# Keep only the necessary columns
region_df = region_df[['ISO3166-1-Alpha-3', 'Region Name']]
region_df.columns = ['ISO3', 'region']

# Merge the region info
df = df_final.merge(region_df, on='ISO3', how='left')

# Custom ISO3 to subregion mapping
iso3_to_subregion = {
    # Americas
    "USA": "North America", "CAN": "North America", "MEX": "North America",
    "BLZ": "Central America", "GTM": "Central America", "HND": "Central America", 
    "SLV": "Central America", "NIC": "Central America", "CRI": "Central America", 
    "PAN": "Central America",
    "ARG": "South America", "BOL": "South America", "BRA": "South America", 
    "CHL": "South America", "COL": "South America", "ECU": "South America", 
    "GUY": "South America", "PRY": "South America", "PER": "South America", 
    "SUR": "South America", "URY": "South America", "VEN": "South America",
    "JAM": "North America", 
    
    # Africa
    "DZA": "North Africa", "EGY": "North Africa", "LBY": "North Africa", 
    "MAR": "North Africa", "SDN": "North Africa", "TUN": "North Africa",
    "AGO": "Central Africa", "CMR": "Central Africa", "CAF": "Central Africa", 
    "COD": "Central Africa", "COG": "Central Africa", "GNQ": "Central Africa", 
    "GAB": "Central Africa", "STP": "Central Africa",
    "ZAF": "Southern Africa", "NAM": "Southern Africa", "BWA": "Southern Africa", 
    "LSO": "Southern Africa", "SWZ": "Southern Africa", "ZMB": "Southern Africa", 
    "MWI": "Southern Africa", "MOZ": "Southern Africa", "ZWE": "Southern Africa",
    "NER": "West Africa", "NGA": "West Africa", "SEN": "West Africa", 
    "MLI": "West Africa", "GHA": "West Africa", "CIV": "West Africa", 
    "BFA": "West Africa", "GIN": "West Africa", "TGO": "West Africa", 
    "BEN": "West Africa", "SLE": "West Africa", "LBR": "West Africa", 
    "GMB": "West Africa", "CPV": "West Africa", "KEN": "Central Africa",
    "MDG": "Southern Africa", "MUS": "Southern Africa", "TZA": "Central Africa",
    "UGA": "Central Africa",

    # Europe
    "ALB": "Southern Europe", "AND": "Southern Europe", "BIH": "Southern Europe",
    "HRV": "Southern Europe", "GRC": "Southern Europe", "ITA": "Southern Europe", 
    "MLT": "Southern Europe", "MNE": "Southern Europe", "MKD": "Southern Europe", 
    "PRT": "Southern Europe", "SMR": "Southern Europe", "SRB": "Southern Europe", 
    "SVN": "Southern Europe", "ESP": "Southern Europe",
    "AUT": "Western Europe", "BEL": "Western Europe", "FRA": "Western Europe", 
    "DEU": "Western Europe", "LUX": "Western Europe", "MCO": "Western Europe", 
    "NLD": "Western Europe", "CHE": "Western Europe",
    "CYP": "Eastern Europe", "CZE": "Eastern Europe", "EST": "Eastern Europe", 
    "HUN": "Eastern Europe", "LVA": "Eastern Europe", "LTU": "Eastern Europe", 
    "POL": "Eastern Europe", "SVK": "Eastern Europe", "UKR": "Eastern Europe", 
    "ROU": "Eastern Europe", "BGR": "Eastern Europe", "RUS": "Eastern Europe", 
    "MDA": "Eastern Europe", "ARM": "Eastern Europe", "GEO": "Eastern Europe",
    "DNK": "Northern Europe", "FIN": "Northern Europe", "ISL": "Northern Europe", 
    "IRL": "Northern Europe", "NOR": "Northern Europe", "SWE": "Northern Europe", 
    "GBR": "Northern Europe",

    # Asia
    "CHN": "East Asia", "JPN": "East Asia", "KOR": "East Asia", "MNG": "East Asia", 
    "TWN": "East Asia", "HKG": "East Asia",
    "IND": "South Asia", "PAK": "South Asia", "BGD": "South Asia", "LKA": "South Asia", 
    "NPL": "South Asia", "BTN": "South Asia", "MDV": "South Asia",
    "THA": "Southeast Asia", "VNM": "Southeast Asia", "IDN": "Southeast Asia", 
    "PHL": "Southeast Asia", "MYS": "Southeast Asia", "SGP": "Southeast Asia", 
    "MMR": "Southeast Asia", "KHM": "Southeast Asia", "LAO": "Southeast Asia", 
    "BRN": "Southeast Asia", "TLS": "Southeast Asia",
    "KAZ": "Central Asia", "UZB": "Central Asia", "TKM": "Central Asia", 
    "KGZ": "Central Asia", "TJK": "Central Asia",
    "TUR": "Middle East", "IRN": "Middle East", "IRQ": "Middle East", "ISR": "Middle East", 
    "SAU": "Middle East", "JOR": "Middle East", "SYR": "Middle East", "LBN": "Middle East", 
    "OMN": "Middle East", "QAT": "Middle East", "ARE": "Middle East", "KWT": "Middle East", 
    "BHR": "Middle East", "YEM": "Middle East", "AZE": "Middle East",

    # Oceania
    "AUS": "Oceania", "NZL": "Oceania", 
    "FJI": "Oceania", "PNG": "Oceania", "SLB": "Oceania", 
    "WSM": "Oceania", "TON": "Oceania", "VUT": "Oceania"
}

# Add subregion column using the mapping
df['subregion'] = df['ISO3'].map(iso3_to_subregion)

# Output to CSV
output_file_path = "final_merged_regions.csv"
df.to_csv(output_file_path, index=False)


 No duplicates found in Tech Data.

 No duplicates found in Health Data.

 No duplicates found in Education Data.

 Duplicate ISO3 codes found in Crime Data:
    ISO3                             Country
145  GBR  United Kingdom (England and Wales)
146  GBR   United Kingdom (Northern Ireland)
147  GBR           United Kingdom (Scotland)


 Duplicate ISO3 codes found in Income Data:
   ISO3        Country
40  CHN          China
41  CHN  China (rural)
42  CHN  China (urban)


Missing ISO3 mappings:
                       Country
28                      Kosovo
30    Taiwan Province of China
70                      Russia
84   Hong Kong S.A.R. of China
86         Congo (Brazzaville)
93                 Ivory Coast
95                     Turkiye
135           Congo (Kinshasa)


In [4]:
import pandas as pd

# Load datasets
scatter_df = pd.read_csv("scatterplot.csv")
regions_df = pd.read_csv("final_merged_regions.csv")

# Standardize country names for better merging
country_name_mapping = {
    "Korea, Republic of": "South Korea",
    "Moldova, Republic of": "Moldova",
    "Russian Federation": "Russia",
    "Tanzania, United Republic of": "Tanzania",
    "Türkiye": "Turkiye"
}
regions_df["Country_Name"] = regions_df["Country_Name"].replace(country_name_mapping)

# Merge datasets on country name
merged_df = regions_df.merge(
    scatter_df,
    how="left",
    left_on="Country_Name",
    right_on="Country"
)

# Drop redundant 'Country' column from scatter_df
merged_df.drop(columns=["Country"], inplace=True)

# Fix Ladder Score column from scatter_df
merged_df.drop(columns=["Ladder Score_x"], inplace=True)
merged_df.rename(columns={"Ladder Score_y": "Ladder Score"}, inplace=True)

# Reorder columns
happiness_columns = [
    'Ladder Score', 'Log GDP per capita', 'Social Support',
    'Healthy Life Expectancy', 'Personal Freedom', 'Generosity',
    'Perceptions of Corruption', 'Dystopia'
]

additional_columns = [
    'Tech', 'Health', 'Education', 'Violence', 'Inequality'
]

ordered_columns = (
    ['Country_Name', 'ISO3', 'region', 'subregion'] +
    happiness_columns + additional_columns +
    [col for col in merged_df.columns if col not in happiness_columns + additional_columns + ['Country_Name', 'ISO3', 'region', 'subregion']]
)

merged_df = merged_df[ordered_columns]

# Save the final dataset
merged_df.to_csv("merged_boxplot_data_cleaned.csv", index=False)

